In [13]:
!pip install python-dotenv

import requests
import json
from supabase import create_client, Client
from datetime import datetime
import pytz
from letta_client import Letta
import time
import os
from dotenv import load_dotenv

load_dotenv()

VAPI_API_URL = os.getenv("VAPI_API_URL")
VAPI_API_KEY = os.getenv("VAPI_API_KEY")
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
LETTA_API_KEY = os.getenv("LETTA_API_KEY")
AGENT_ID = os.getenv("AGENT_ID")




def get_first_call_structured_data():
    try:
        response = requests.get(
            VAPI_API_URL,
            headers={"Authorization": f"Bearer {VAPI_API_KEY}"}
        )
        response.raise_for_status()
        call_log = response.json()

        if not isinstance(call_log, list) or not call_log:
            print("No calls found or invalid response format.")
            return None, None

        first_call = call_log[0]
        call_id = first_call.get('id', 'Unknown ID')
        analysis = first_call.get('analysis', {})
        structured_data = analysis.get('structuredData', {})

        return call_id, structured_data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching call logs: {e}")
        return None, None
    except Exception as e:
        print(f"Error processing response: {e}")
        return None, None

def process_new_call():
    # Get call data
    call_id, structured_data = get_first_call_structured_data()
    
    if call_id and structured_data:
        print(f"First Call ID: {call_id}")
        print("Structured Data:")
        print(json.dumps(structured_data, indent=2))
    else:
        print("Failed to retrieve structured data.")
        return False

    # Prepare data for Supabase
    insert_data = {
        "id": call_id,
        "created_at": datetime.now(pytz.UTC).isoformat(),
        "data": structured_data
    }
    print("Insert Data:", json.dumps(insert_data, indent=2))

    # Initialize Supabase client
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    
    # Insert into Supabase
    response = supabase.table('transcriptions').insert(insert_data).execute()
    if not response.data:
        raise Exception(f"Supabase insert error: {response.error}")
    print("Data inserted successfully:", response.data)

    # Initialize Letta client
    client = Letta(token=LETTA_API_KEY)
    
    # Get the latest entry from Supabase
    response = supabase.table('transcriptions').select('*').order('created_at', desc=True).limit(1).execute()
    first_entry = response.data[0]
    data_json = first_entry['data']
    
    print("Data JSON to Query Letta:", json.dumps(data_json, indent=2))
    message_content = json.dumps(data_json)

    # Send to Letta agent
    response = client.agents.messages.create(
        agent_id=AGENT_ID,
        messages=[{"role": "user", "content": message_content}]
    )
    print("Letta response:", response.messages)
    return True

def main():
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    last_call_id = None
    
    while True:
        try:
            # Get the latest entry
            response = supabase.table('transcriptions').select('id', count='exact').order('created_at', desc=True).limit(1).execute()
            
            if response.data:
                current_call_id = response.data[0]['id']
                
                # Check if the first call ID has changed
                if current_call_id != last_call_id:
                    print(f"New call detected: {current_call_id}")
                    if process_new_call():
                        last_call_id = current_call_id
                else:
                    print("No new calls detected.")
            else:
                print("No entries in database yet.")
            
            # Wait for 3 minutes (180 seconds)
            time.sleep(10)
            
        except Exception as e:
            print(f"Error in main loop: {e}")
            time.sleep(10)  # Wait before retrying in case of error

if __name__ == "__main__":
    main()

python-dotenv could not parse statement starting at line 1
python-dotenv could not parse statement starting at line 9


New call detected: 6af410dc-fbee-4a9a-aded-b595890f42a0
First Call ID: 6af410dc-fbee-4a9a-aded-b595890f42a0
Structured Data:
{
  "misc": "We gave him NRB at 15 liters per minute. He's only responsive to painful stimuli.",
  "heart_rate": "49 and irregular",
  "patient_age": 62,
  "patient_sex": "Male",
  "dispatch_time": "14:30",
  "initial_report": "we found him sitting on the ground clutching his chest very pale and sweating. He was barely conscious.",
  "vehicle_number": "EMS 12",
  "ems_officer_name": "Amy Davis",
  "incident_location": "Golden Gate Park",
  "oxygen_saturation": "88 percent of room air",
  "patient_full_name": "Andrew Chen",
  "treatment_urgency": "Critical",
  "level_of_consciousness": "barely conscious"
}
Insert Data: {
  "id": "6af410dc-fbee-4a9a-aded-b595890f42a0",
  "created_at": "2025-06-22T17:55:32.119896+00:00",
  "data": {
    "misc": "We gave him NRB at 15 liters per minute. He's only responsive to painful stimuli.",
    "heart_rate": "49 and irregular",
